In [5]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random

# Generate realistic motor temperature dataset (100 entries)
start_time = datetime(2025, 4, 6, 10, 0, 0)
timestamps = [start_time + timedelta(seconds=10 * i) for i in range(1000)]

temperatures = []
loads = []
rpms = []
statuses = []

for _ in range(1000):
    zone = random.choices(['Healthy', 'Caution', 'Failure Likely'], weights=[0.5, 0.3, 0.2])[0]
    
    if zone == 'Healthy':
        temp = round(random.uniform(30, 59), 1)
    elif zone == 'Caution':
        temp = round(random.uniform(60, 79), 1)
    else:
        temp = round(random.uniform(80, 90), 1)
    
    load = random.randint(60, 95)  # Motor load in percentage
    rpm = random.randint(1450, 1500)  # Typical industrial motor speed
    
    temperatures.append(temp)
    loads.append(load)
    rpms.append(rpm)
    statuses.append(zone)

# Create DataFrame
df = pd.DataFrame({
    "Temperature (°C)": temperatures,
    "Status": statuses
})

# Save to CSV
csv_path = "Industrial_Motor_Temperature_Data.csv"
df.to_csv(csv_path, index=False)

csv_path


'Industrial_Motor_Temperature_Data.csv'

In [6]:
import pandas as pd

# Load your CSV
df = pd.read_csv('Industrial_Motor_Temperature_Data.csv')
print(df.head())


   Temperature (°C)   Status
0              55.8  Healthy
1              38.1  Healthy
2              56.3  Healthy
3              69.1  Caution
4              65.9  Caution


In [8]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# Encode the status column
le = LabelEncoder()
df['Status_encoded'] = le.fit_transform(df['Status'])  # Healthy=0, Caution=1, Failure Likely=2

# Features and labels
X = df[['Temperature (°C)']]
y = df['Status_encoded']

# Split into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [9]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Initialize and train
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Predict on test set
y_pred = model.predict(X_test)


In [10]:
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:")
print(classification_report(y_test, y_pred, target_names=le.classes_))


Accuracy: 1.0
Classification Report:
                precision    recall  f1-score   support

       Caution       1.00      1.00      1.00        63
Failure Likely       1.00      1.00      1.00        45
       Healthy       1.00      1.00      1.00        92

      accuracy                           1.00       200
     macro avg       1.00      1.00      1.00       200
  weighted avg       1.00      1.00      1.00       200



In [14]:
# Example prediction
import numpy as np
new_temp = pd.DataFrame([[78.5]], columns=['Temperature (°C)'])
pred = model.predict(new_temp)
status = le.inverse_transform(pred)
print("Predicted Status:", status[0])


Predicted Status: Caution


In [ ]:
import joblib
joblib.dump(model, 'temperature_model.pkl')
joblib.dump(le, 'label_encoder.pkl')
